## DIM0549 - Grafos : 1º Unidade, Grupo 5

### Classes abstratas contendo metodos basicos das classes *Vertice* e *Grafo*

## Item **1** : Criação do grafo a partir da lista de adjancências
A implementação do grafo que utiliza a lista de adjacência como estrutura de dados subjacente foi feita através das classes *ListaAdjacencia* e *VerticeAdjacencia*.
Aqui, contamos com o método *ler_grafo*, responsável por realizar a leitura do grafo descrito no filepath fornecido como
entrada e fazer a construção do grafo. Essa construção consiste em, para cada par de vértices lidos, instanciar os objetos
vértices correspondentes, e inserí-los no grafo via o método *adicionar_vertice*, caso já não pertençam ao grafo. Se já pertencerem, é feita a
criação de novas arestas via a função *adicionar_vizinho* encapsulada na classe *VerticeAdjacencia*. 
Maior detalhamento sobre o método de adição será fornecido na resolução do item **9**
Esta implementação da classe já conta, também, com a implementação do método de remoção de vértice ao grafo.



In [445]:
from collections import Counter
class VerticeAdjacencia:
    def __init__(self, label, vizinhos = None):
        self.label = label
        # self.predecessor = predecessor if predecessor else None
        vizinhos = vizinhos if vizinhos else []
        # TODO : Remover
        self.vizinhos = set(vizinhos)
    def adicionar_vizinho(self, vizinho):
        self.vizinhos.add(vizinho)
        vizinho.vizinhos.add(self)
    def __str__(self):
        return f"Label : {self.label}"

# Nesta implementacao o grafo consistirá em um dict cujos valores sao lists (implementacoes de linkedlista=)
class ListaAdjacencia:
    def __init__(self, nome,  caminho_arquivo : str = None):
        self.lista = dict()  
        self.labels = dict()
        self.nome = nome
        self.tamanho = 0
        if caminho_arquivo:
            vertices = self.ler_grafo(caminho_arquivo=caminho_arquivo)
            for vertice in vertices:
                self.adicionar_vertice(vertice)

    def adicionar_vertice(self, novo_vertice): # O(m), tq m eh numero de vertices vizinhos ao novo_vertice JA PRESENTES NO GRAFO
        self.lista[novo_vertice] = set()
        self.labels[novo_vertice.label] = novo_vertice
        # Ver os vizinhos do vertice que vai ser adicionado
        # adicionar o novo vertice a lista de colisao de cada um de seus vizinhos 

        for vizinho in novo_vertice.vizinhos:
            # Fazemos a checagem se o a aresta ja foi contabilizada para incrementar o atributo 'tamanho'
            # Para isso, checamos se apos a insercao do vizinho 
            # tamanho_set_antes =  len(self.lista[novo_vertice])
            vizinho.adicionar_vizinho(novo_vertice)
            self.lista[novo_vertice].add(vizinho) 
            if not self.lista.get(vizinho):
                self.lista[vizinho] = set()
            tamanho_antes = len(self.lista[vizinho])
            self.lista[vizinho].add(novo_vertice)
            if tamanho_antes < len(self.lista[vizinho]):
                self.tamanho += 1

    def ler_grafo(self, caminho_arquivo : str):
        with open(caminho_arquivo, "r") as grafo_arquivo:
            linhas = grafo_arquivo.readlines()
            ordem = int(linhas[0])
            vertices = dict()
            def extrair_labels(linha: str):
                linha_split = linha.strip().split(",")
                return linha_split[0], linha_split[1]

            for linha in linhas[1:]:
                label_1, label_2 = extrair_labels(linha)
                # Checa se os vertices recebidos na linha ja existem
                vert_1, vert_2 = vertices.get(label_1, None), vertices.get(label_2, None)
                if vert_1 and vert_2:
                    vert_1.adicionar_vizinho(vert_2)
                elif vert_1:
                    vert_2 = VerticeAdjacencia(label=label_2, vizinhos=[vert_1])
                    vertices[label_2] = vert_2
                elif vert_2:
                    vert_1 = VerticeAdjacencia(label=label_1, vizinhos=[vert_2])
                    vertices[label_1] = vert_1
                else:
                    vert_2 = VerticeAdjacencia(label=label_2)
                    vert_1 = VerticeAdjacencia(label=label_1, vizinhos=[vert_2])
                    vertices[label_1] = vert_1
                    vertices[label_2] = vert_2
            return vertices.values()
    def remover_vertice(self, vertice_a_ser_removido):
        if vertice_a_ser_removido not in self.lista:
            return "Vertice nao pertence ao grafo"
        self.lista.pop(vertice_a_ser_removido)  # O(1)
        for vizinho in vertice_a_ser_removido.vizinhos: # O(m)
            del self.lista[vizinho][vertice_a_ser_removido] # O(1)
            self.tamanho =- 1

    def get_vertices(self):
        return self.lista.keys()

    def get_vertice(self,label):
        return self.labels[label]
    

    def get_adjacentes(self, label_vertice):
        vertice = self.labels[label_vertice]
        return set(vertice.vizinhos)

    def __str__(self):
        if len(self.lista) ==0:
            return "<ListaAdjacencia Vazio>"
        strings = [f"{vertice.label} -> {list(map(lambda vizinho : vizinho.label, self.lista[vertice]))}" for vertice in self.lista.keys()]
        return '\n'.join(strings)


grafo = ListaAdjacencia("Grafo 1",caminho_arquivo="./data/GRAFO_1.txt")
print(grafo)


a -> ['b']
b -> ['a', 'c', 'd', 'e', 'f']
c -> ['e', 'i', 'b', 'd']
e -> ['c', 'b']
d -> ['c', 'b', 'h']
f -> ['b']
i -> ['c']
h -> ['d']


## Item **2** : Implementação do Grafo usando *Matriz de adjacência*
A implementação usando matriz de adjacência tem a mesma estrutura do item anterior, é feita a leitura via método *ler_grafo* e adição dos vértices.
Nesta implementação, mantemos o a classe Vertice, mas aqui ela é usada apenas para encapsular o dado vértice e seus vizinhos, para que sirva de entrada ao método *adicionar_vertice*. A estrutura do grafo é mantida integralmente na matriz

In [446]:
from collections import Counter
from functools import reduce

# Implementacao da classe vertice para de grafos utilizando matriz de adjacencia 
class VerticeMatriz:
    def __init__(self, label : str = "", vizinhos = None):
        self.label = label 
        self.indice = None
        # O Counter associa um objeto a um contador numérico via hash table, aqui será usado para contar o número de
        # arestas entre este vértice e seu respectivo vizinho, para o caso de recebermos multigrafos
        self.vizinhos = Counter() 
        if vizinhos:
            for vizinho in vizinhos:
                self.adicionar_vizinho(vizinho)

    def adicionar_vizinho(self, vertice_vizinho):
        self.vizinhos[vertice_vizinho] += 1
        vertice_vizinho.vizinhos[self] += 1

class MatrizAdjacencia:
    def __init__(self, nome : str ,caminho_arquivo : str = None, vertices = None): # O(n^2)
        self.nome = nome # Nome do grafo, pra proposito de impressao
        self.indices = dict() # hash table associando os vertices a cada indice da matriz, para agilizar consultas
        self.matriz = [] 
        self.labels = dict() # Hash table para armazenar os vértices associadas a cada label
        self.tamanho = 0
        if vertices:
            for vertice in vertices:
                self.adicionar_vertice(vertice)
        else:
            if caminho_arquivo:
                for vertice in self.ler_grafo(caminho_arquivo):
                    self.adicionar_vertice(vertice)

    def ler_grafo(self, caminho_arquivo : str):
        with open(caminho_arquivo, "r") as grafo_arquivo:
            linhas = grafo_arquivo.readlines()
            ordem = int(linhas[0])
            vertices = dict()
            def extrair_labels(linha: str):
                label1, label2 = linha.strip().split(",")
                if label1.isnumeric():
                    label1 = int(label1)
                    label2 = int(label2)
                return label1, label2

            for linha in linhas[1:]:
                label_1, label_2 = extrair_labels(linha)

                # Checa se os vertices recebidos na linha ja existem
                vert_1, vert_2 = vertices.get(label_1, None), vertices.get(label_2, None)
                if vert_1 and vert_2:
                    vert_1.adicionar_vizinho(vert_2)
                elif vert_1:
                    vert_2 = VerticeMatriz(label=label_2)
                    vert_1.adicionar_vizinho(vert_2)
                    vertices[label_2] = vert_2
                elif vert_2:
                    vert_1 = VerticeMatriz(label=label_1, vizinhos=[vert_2])
                    vertices[label_1] = vert_1
                else:
                    vert_2 = VerticeMatriz(label=label_2)
                    vert_1 = VerticeMatriz(label=label_1, vizinhos=[vert_2])
                    vertices[label_1] = vert_1
                    vertices[label_2] = vert_2
            return sorted(vertices.values(), key=lambda key : key.label)


    def adicionar_vertice(self, vertice : VerticeMatriz): # O(n)
        # Adiciona o novo label
        self.labels[vertice.label] = vertice
        # Caso a matriz esteja vazia
        if len(self.matriz) == 0:
            vertice.indice = 0
            # Adicionamos à linha o valor referente ao número de laços presentes neste vértice 
            self.matriz.append(list([vertice.vizinhos[vertice]]))
            self.indices[0] = vertice
            self.tamanho += vertice.vizinhos[vertice]
            return

        # Linha que será adcioanada à matriz
        nova_linha = []
        # Adiciona o índice do novo vértice à hash table
        self.indices[len(self.matriz)] = vertice # O(1)
        vertice.indice = len(self.matriz) # O(1)
        # Itera por cada linha da matriz adicionando uma nova coluna com valor igual ao número de arestas entre
        # o novo vértice e o da linha atual
        for indice, linha in enumerate(self.matriz): # O(n)
            vertice_atual = self.indices[indice] # O(1)
            quantidade = vertice.vizinhos.get(vertice_atual, 0) # O(1)
            linha.append(quantidade) # O(1)
            nova_linha.append(quantidade) #O(1)
            self.tamanho += quantidade

        # Atualiza a nova linha para o caso do vértice adicionado apresentar laços
        quantidade_arestas_proprias = vertice.vizinhos.get(vertice, 0) # O(1)
        self.tamanho += quantidade_arestas_proprias
        nova_linha.append(quantidade_arestas_proprias) # O(1)
        self.matriz.append(nova_linha) #O(1)

    def get_vertices(self):
        return self.indices.values()

    def get_adjacentes(self, label_vertice):
        vertice = self.labels[label_vertice]
        indice_na_matriz = vertice.indice
        linha = self.matriz[indice_na_matriz]
        # É feita a filtragem dos elementos 
        vizinhos_map = map(lambda i : self.indices[i], filter(lambda indice : linha[indice] > 0, range(0, len(linha))))
        return set(vizinhos_map)

    def get_vertice(self, label_vertice):
        return self.labels[label_vertice]

    def remover_vertice(self, label_vertice): # O(n + n*n) = O(n^2)
        if label_vertice not in self.labels:
            raise ValueError("Vértice não faz parte do grafo") # O(1)

        vertice = self.labels[label_vertice]
        indice_a_remover = vertice.indice
        print(self.indices.keys())
        for i, linha in enumerate(self.matriz): # O(n)
            if i == indice_a_remover:
                continue
            if linha[indice_a_remover] != 0:    # O(1)
                # Remove o vertice removido da lista de vizinhos dos outros vertices
                self.indices[i].vizinhos.pop(vertice) # O(1)
                self.tamanho -= 1
            # Atualiza os valores dos indices associados aos vertices para refletir as novas posicoes
            if i > indice_a_remover: 
                self.indices[i - 1] = self.indices[i]
                self.indices[i - 1].indice = i - 1
                self.indices.pop(i)
            linha.pop(indice_a_remover) # O(n)

        # Remove da matriz a linha referente ao vértice removido
        self.matriz.pop(indice_a_remover)

    def __str__(self):
        return str(self.matriz)

In [447]:
grafo = MatrizAdjacencia("Grafo - 1", "./data/GRAFO_2.txt")
print(grafo)

['0 -> 1', '1 -> 2', '2 -> 3', '3 -> 4', '4 -> 5', '5 -> 6', '6 -> 7', '7 -> 8', '8 -> 9', '9 -> 10', '10 -> 11']
[[0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0], [1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0], [1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0], [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]]


## Item **3** - Implementação do Grafo usando da Matriz de Incidência 

In [448]:
class MatrizIncidencia:
    def __init__(self, nome, caminho_arquivo=None):
        self.nome = nome # para melhor identificação 
        self.vertices = {}  # hash table para mapear label do vértice para o índice
        self.arestas = []   # hash table representando as arestas
        self.matriz = []    

        if caminho_arquivo:
            self.ler_grafo(caminho_arquivo)

    def ler_grafo(self, caminho_arquivo):
        with open(caminho_arquivo, "r") as f:
            linhas = f.readlines()[1:] # para ignorar a primeira linha com o número de vértices

            # Primeiro, identifica todos os vértices únicos para definir o tamanho da matriz
            vertices_labels = set()
            for linha in linhas:
                v1_label, v2_label = linha.strip().split(',')
                vertices_labels.add(v1_label)
                vertices_labels.add(v2_label)
            
            for label in sorted(list(vertices_labels)): # ordena para consistência
                self.adicionar_vertice(label)

            # adiciona as arestas e constrói a matriz
            for linha in linhas:
                v1_label, v2_label = linha.strip().split(',')
                self.adicionar_aresta(v1_label, v2_label)

    def adicionar_vertice(self, label):
        if label not in self.vertices:
            self.vertices[label] = len(self.vertices)
            # adiciona uma nova linha (vazia por enquanto) para o novo vértice
            self.matriz.append([])

    def adicionar_aresta(self, v1_label, v2_label):
        # adiciona a nova aresta
        self.arestas.append((v1_label, v2_label))
        indice_aresta = len(self.arestas) - 1
        
        # adiciona a nova coluna à matriz (representada por adicionar um elemento a cada linha)
        for i in range(len(self.vertices)):
            self.matriz[i].append(0)

        # preenche os valores da nova coluna
        i1 = self.vertices[v1_label]
        i2 = self.vertices[v2_label]
        
        if i1 == i2:  # laço
            self.matriz[i1][indice_aresta] = 2
        else:  # grafo não direcionado
            self.matriz[i1][indice_aresta] = 1
            self.matriz[i2][indice_aresta] = 1

    def __str__(self):
       return str(self.matriz)

grafo_incidencia = MatrizIncidencia("GRAFO 1", "./data/GRAFO_1.txt")
print(grafo_incidencia)

[[1, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 0, 0, 0, 0], [0, 1, 0, 0, 0, 1, 1, 1, 0], [0, 0, 1, 0, 0, 1, 0, 0, 1], [0, 0, 0, 1, 0, 0, 1, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 1, 0]]


## Item **4** : Conversão de matriz de adjacência para lista de Adjacências e vice-versa.
A implementação é bastante simples: obtemos os vértices do grafo inicial a partir do método *get_vertices* e 
fazemos a inserção destes no novo grafo vazio - que utiliza a estrutura de dados procurada.

In [ ]:
def converter_lista_pra_matriz(grafo_inicial : ListaAdjacencia) -> MatrizAdjacencia:
    
    vertices_convertidos = dict()
    for vertice in grafo_inicial.get_vertices():
        vertice_convertido = VerticeMatriz(label=vertice.label)
        for vizinho in vertice.vizinhos:
            vizinho_convertido = vertices_convertidos.get(vizinho.label, None)
            if not vizinho_convertido:
                vizinho_convertido = VerticeMatriz(label=vizinho.label)
                vertices_convertidos[vizinho.label] = vizinho_convertido
            vertice_convertido.adicionar_vizinho(vizinho_convertido)
            # vizinho_convertido.adicionar_vizinho(vertice_convertido)
        vertices_convertidos[vertice_convertido.label] = vertice_convertido
    novo = MatrizAdjacencia(nome=grafo_inicial.nome, vertices=sorted(list(vertices_convertidos.values()), key=lambda key : key.label))
    return  novo
        
def converter_matriz_pra_lista(grafo_inicial : MatrizAdjacencia) -> ListaAdjacencia:
    novo_grafo = ListaAdjacencia(grafo_inicial.nome)
    for vertice in grafo_inicial.get_vertices():
        novo_grafo.adicionar_vertice(vertice)
    return novo_grafo

matriz = MatrizAdjacencia(nome="Grafo que sera convertido", caminho_arquivo="./data/GRAFO_2.txt")
print(matriz)
lista = converter_matriz_pra_lista(matriz)
print(converter_matriz_pra_lista(matriz))
matriz = converter_lista_pra_matriz(lista)
print(converter_lista_pra_matriz(lista))


[[0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0], [1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0], [1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0], [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]]
1 -> [3, 6, 2]
2 -> [3, 1, 6, 5]
3 -> [4, 1, 2]
6 -> [1, 7, 2]
5 -> [4, 2]
4 -> [3, 5]
7 -> [6]
8 -> [11, 9, 10]
9 -> [10, 8]
10 -> [9, 8]
11 -> [8]
[[0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0], [1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0], [1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0], [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]]


## Item **5** : Função que calcula o grau de cada vértice.

In [450]:
from functools import reduce
def calcular_grau_vertice_matriz(grafo, label_vertice) -> int:
    # Obtem o objeto do vertice de entrada, para que possamos extrair seu indice na matriz
    vertice = grafo.get_vertice(label_vertice)
    # Faz uma operacao reducao somando todos os valores da linha correspondente ao vertice de entrada
    # Isso nos fornece o numero de arestas no qual este vertice eh terminal, e.g seu grau
    return reduce(lambda a,b : a+b, grafo.matriz[vertice.indice])
grafo = MatrizAdjacencia("Grafo", "./data/GRAFO_2.txt")
print(calcular_grau_vertice_matriz(grafo, 3))


3


## Item **6** : Função que determina se dois vértices são adjacentes.

In [451]:
# Implementacao para a Matriz e Lista de adjacencia
def sao_adjacentes(grafo, label_vert_1, label_vert_2) -> bool : # O(1)
    vert_1, vert_2 = grafo.get_vertice(label_vert_1) , grafo.get_vertice(label_vert_2) # O(1)
    return vert_2 in vert_1.vizinhos or vert_1 in vert_2.vizinhos


grafo = MatrizAdjacencia("Grafo", "./data/GRAFO_2.txt")
print(sao_adjacentes(grafo, 1, 10))


False


## Itens **7** e **8** : Função que determina os números totais de vértices e arestas
Para obter a ordem dos grafos implementados com lista de adjacência, basta obter o tamanho do seu atributo *lista*, já para a matriz, o tamanho de *matriz*.
A obtenção do tamanho para os grafos nessas estruturas é um pouco mais complexa. A solução ingênua seria: para a lista, iterar por cada vértice e incrementar o tamanho para cada vizinho encontrado, enquanto o para a matriz basta checar as arestas em cada linha.
Para dirimir a complexidade de invocar este método, o atributo tamanho é ajustado a cada chamada dos métodos *adicionar_vertice* e *remover_vertice*.
Para a lista, ao adicionar vértice, é checado se as novas arestas conectadas a ele já foram contabilizadas - no caso de seu vértice adjacente conectado a esta aresta já ter sido adicionado - se não, incrementao tamanho

In [452]:
def get_tamanho(grafo):
    return grafo.tamanho


def get_ordem_lista(grafo):
    return len(grafo.lista)

def get_ordem_matriz_adjacencia(grafo):
    return len(grafo.matriz)

MatrizAdjacencia.get_ordem = get_ordem_matriz_adjacencia
ListaAdjacencia.get_ordem = get_ordem_lista

MatrizAdjacencia.get_tamanho = get_tamanho
ListaAdjacencia.get_tamanho = get_tamanho
grafo = ListaAdjacencia("Grafo 1 - Lista de Adjacencia", "./data/GRAFO_2.txt")
print(grafo)
print(f"Ordem do grafo ({grafo.nome}): {grafo.get_ordem()}")
print(f"Tamanho do grafo ({grafo.nome}): {grafo.get_tamanho()}")

grafo_mat = MatrizAdjacencia("Grafo 1 - Matriz de Adjacencia", "./data/GRAFO_2.txt")
print(f"Ordem do grafo ({grafo_mat.nome}): {grafo_mat.get_ordem()}")
print(f"Tamanho do grafo ({grafo_mat.nome}): {grafo_mat.get_tamanho()}")


1 -> ['6', '3', '2']
6 -> ['1', '7', '2']
2 -> ['1', '3', '6', '5']
3 -> ['1', '4', '2']
5 -> ['4', '2']
4 -> ['3', '5']
7 -> ['6']
8 -> ['11', '9', '10']
9 -> ['8', '10']
10 -> ['8', '9']
11 -> ['8']
Ordem do grafo (Grafo 1 - Lista de Adjacencia): 11
Tamanho do grafo (Grafo 1 - Lista de Adjacencia): 13
Ordem do grafo (Grafo 1 - Matriz de Adjacencia): 11
Tamanho do grafo (Grafo 1 - Matriz de Adjacencia): 13


## Item **14** - Busca em Profundidade, com determinação de arestas de retorno, a partir de um vértice em específico

In [453]:
# TODO : Retornar árvore de profundidade e checar se precisa fazer o percurso na ordem
def depth_first_search(grafo, label_vertice_inicial, ordenado : bool = None):
    pilha = list()
    vertice_inicial = grafo.get_vertice(label_vertice_inicial)
    # AS aresta serao representadas por 2-uplas
    arestas_visitadas = set()
    arestas_de_retorno = set()
    # Set para monitorar quais vertices ainda nao foram visitados
    vertices_nao_visitados = set(grafo.get_vertices())
    # Set para monitorar quais vertices ainda nao foram explorados
    vertices_nao_explorados = set(grafo.get_vertices()) 
    print(f"## Percurso em profundidade em '{grafo.nome}'") 
    print(f"Vertice visitado: {vertice_inicial.label}, predecessor: X", )
    pilha.append(vertice_inicial)
    vertices_nao_visitados.remove(vertice_inicial)
    vizinhos_vertices = dict()
    while vertices_nao_explorados:
        if len(pilha) == 0:
            menor_label = min([i.label for i in vertices_nao_visitados])
            nova_raiz = grafo.get_vertice(min([i.label for i in vertices_nao_visitados]))
            pilha.append(nova_raiz)
            print(f"Vertice visitado: {nova_raiz.label}, predecessor: X", )
            vertices_nao_visitados.remove(nova_raiz)
        u = pilha[-1]
        if u not in vizinhos_vertices:
            if ordenado:
                # TODO : Refatorar
                labels_ordenadas = sorted([i.label for i in grafo.get_adjacentes(u.label)], reverse=True)
                vizinhos_vertices[u] = [grafo.get_vertice(v) for v in labels_ordenadas]
            else:
                vizinhos_vertices[u] = grafo.get_adjacentes(u.label)
        
        v = vizinhos_vertices[u].pop() if len(vizinhos_vertices[u]) > 0 else None
        if not v:
            pilha.pop()
            vertices_nao_explorados.remove(u) # O vertice eh marcado como explorado
        elif v in vertices_nao_visitados:
            vertices_nao_visitados.remove(v)
            print(f"Vertice visitado: {v.label}, predecessor : {u.label}", )
            pilha.append(v)
            arestas_visitadas.add((u,v))
        else:
            if v in vertices_nao_explorados and pilha[-2] != v: # Checa se é adjaccente na pilha
                if (v,u) not in arestas_visitadas:
                    arestas_visitadas.add((u,v))
                    arestas_de_retorno.add((u,v))
                    print("Nova aresta de retorno: ", (u.label,v.label))

    # Printa arestas de retorno
    arestas_de_retorno = map(lambda aresta : (aresta[0].label, aresta[1].label), arestas_de_retorno)
    print(f"Arestas de retorno: {list(arestas_de_retorno)}")

In [454]:
from IPython.display import display_markdown
display_markdown("**Busca em profundidade em grafo conexo**")
grafo1 = MatrizAdjacencia("Grafo Matriz", "./data/GRAFO_1.txt")
depth_first_search(grafo=grafo1, label_vertice_inicial="a", ordenado=True)

grafo1 = ListaAdjacencia("Grafo Lista", "./data/GRAFO_1.txt")
depth_first_search(grafo=grafo1, label_vertice_inicial="a", ordenado=True)



## Percurso em profundidade em 'Grafo Matriz'
Vertice visitado: a, predecessor: X
Vertice visitado: b, predecessor : a
Vertice visitado: c, predecessor : b
Vertice visitado: d, predecessor : c
Nova aresta de retorno:  ('d', 'b')
Vertice visitado: h, predecessor : d
Vertice visitado: e, predecessor : c
Nova aresta de retorno:  ('e', 'b')
Vertice visitado: i, predecessor : c
Vertice visitado: f, predecessor : b
Arestas de retorno: [('e', 'b'), ('d', 'b')]
## Percurso em profundidade em 'Grafo Lista'
Vertice visitado: a, predecessor: X
Vertice visitado: b, predecessor : a
Vertice visitado: c, predecessor : b
Vertice visitado: d, predecessor : c
Nova aresta de retorno:  ('d', 'b')
Vertice visitado: h, predecessor : d
Vertice visitado: e, predecessor : c
Nova aresta de retorno:  ('e', 'b')
Vertice visitado: i, predecessor : c
Vertice visitado: f, predecessor : b
Arestas de retorno: [('e', 'b'), ('d', 'b')]


In [455]:

display_markdown("**Busca em profundidade em grafo nao conexo**")
grafo1 = ListaAdjacencia("Grafo 2 - Lista de Adjacencia", "./data/GRAFO_2.txt")
depth_first_search(grafo=grafo1, label_vertice_inicial="1", ordenado=True)

grafo1 = MatrizAdjacencia("Grafo 2 - Matriz de Adjacencia", "./data/GRAFO_2.txt")
depth_first_search(grafo=grafo1, label_vertice_inicial=1, ordenado=True)

## Percurso em profundidade em 'Grafo 2 - Lista de Adjacencia'
Vertice visitado: 1, predecessor: X
Vertice visitado: 2, predecessor : 1
Vertice visitado: 3, predecessor : 2
Nova aresta de retorno:  ('3', '1')
Vertice visitado: 4, predecessor : 3
Vertice visitado: 5, predecessor : 4
Nova aresta de retorno:  ('5', '2')
Vertice visitado: 6, predecessor : 2
Nova aresta de retorno:  ('6', '1')
Vertice visitado: 7, predecessor : 6
Vertice visitado: 10, predecessor: X
Vertice visitado: 8, predecessor : 10
Vertice visitado: 11, predecessor : 8
Vertice visitado: 9, predecessor : 8
Nova aresta de retorno:  ('9', '10')
Arestas de retorno: [('6', '1'), ('5', '2'), ('9', '10'), ('3', '1')]
## Percurso em profundidade em 'Grafo 2 - Matriz de Adjacencia'
Vertice visitado: 1, predecessor: X
Vertice visitado: 2, predecessor : 1
Vertice visitado: 3, predecessor : 2
Nova aresta de retorno:  (3, 1)
Vertice visitado: 4, predecessor : 3
Vertice visitado: 5, predecessor : 4
Nova aresta de retorno:  (5, 2)
Ve

## Item **13** - Busca em Largura, a partir de um vértice específico.

In [456]:
from collections import deque
def width_first_search(grafo, label_vertice_inicial, ordenado : bool = None):
    fila = deque()
    vertices_nao_visitados = set(grafo.get_vertices()) # Pra o caso do grafo nao ser conexo
    arestas_visitadas = set()
    vertice_inicial = grafo.get_vertice(label_vertice_inicial)
    vertices_nao_visitados.remove(vertice_inicial)
    print("## Percurso em Largura")
    print("Vertice inicial: ", vertice_inicial.label)
    fila.append(vertice_inicial)
    vizinhos_vertices = dict()
    eh_conexo = False
    while len(vertices_nao_visitados) > 0:
        if len(fila) == 0:
            nova_raiz =vertices_nao_visitados.pop()
            print(f"Vertice visitado: {nova_raiz.label}; Predecessor : X",)
            fila.append(nova_raiz)
            eh_conexo = True
        u = fila.popleft()
        # Se os vertices adjacentes de u nao estiverem em nosso dict que os armazena, os insere
        if ordenado:
            labels_ordenadas = sorted([i.label for i in grafo.get_adjacentes(u.label)], reverse=True)
            vizinhos_vertices[u] = [grafo.get_vertice(v) for v in labels_ordenadas]
        else:
            vizinhos_vertices[u] = grafo.get_adjacentes(u.label)
        v = vizinhos_vertices[u].pop() if len(vizinhos_vertices[u]) > 0 else None
        while v: # O(n) -> n : numero de vizinhos do vertice u
            if v in vertices_nao_visitados:
                vertices_nao_visitados.remove(v)
                fila.append(v)
                print(f"Vertice visitado: {v.label}; Predecessor : {u.label}",)
            v = vizinhos_vertices[u].pop() if len(vizinhos_vertices[u]) > 0 else None

In [457]:
grafo_matriz = ListaAdjacencia("Grafo Matriz", "./data/GRAFO_2.txt")
print(grafo_matriz)
width_first_search(grafo_matriz, "1")

1 -> ['2', '6', '3']
2 -> ['5', '6', '1', '3']
6 -> ['2', '1', '7']
3 -> ['2', '4', '1']
5 -> ['2', '4']
4 -> ['5', '3']
7 -> ['6']
8 -> ['10', '9', '11']
9 -> ['10', '8']
10 -> ['9', '8']
11 -> ['8']
## Percurso em Largura
Vertice inicial:  1
Vertice visitado: 2; Predecessor : 1
Vertice visitado: 6; Predecessor : 1
Vertice visitado: 3; Predecessor : 1
Vertice visitado: 5; Predecessor : 2
Vertice visitado: 7; Predecessor : 6
Vertice visitado: 4; Predecessor : 3
Vertice visitado: 9; Predecessor : X
Vertice visitado: 10; Predecessor : 9
Vertice visitado: 8; Predecessor : 9
Vertice visitado: 11; Predecessor : 8


## Item **12** (Opcional) - Determinar se um grafo é bipartido
Para implementar essa função, foi usada a estratégia extraída da seção *Testing bipartiteness* no endereço *https://en.wikipedia.org/wiki/Bipartite_graph*.
O procedimento consiste em realizar um percurso em profundidade colorindo o grafo em duas cores - na implementação, as cores são representadas por 1 e -1. Caso seja possível colorir cada vértice com a cor oposta do seu vértice pai na árvore, o grafo é bipartido e a função retorna True, caso contrário, não é bipartido e retornamos False.

In [458]:
def eh_bipartido(grafo):
    # Pega o vertice inicial
    vertice_inicial = list(grafo.get_vertices())[0]
    pilha = list()
    # Map para armazenar a cor de cada grafo
    vertices_coloridos = dict()
    # AS aresta serao representadas por 2-uplas
    arestas_visitadas = set()
    arestas_de_retorno = set()
    # Set para monitorar quais vertices ainda nao foram visitados
    vertices_nao_visitados = set(grafo.get_vertices())
    # Set para monitorar quais vertices ainda nao foram explorados
    vertices_nao_explorados = set(grafo.get_vertices()) 
    # Empilha o vértice inicial
    pilha.append(vertice_inicial)
    vertices_nao_visitados.remove(vertice_inicial)
    vertices_coloridos[vertice_inicial.label] = 1
    vizinhos_vertices = dict()
    # TODO : Essa variável é utilizada para armazenar a cor do último vértice visitado, para que em casos de grafos não
    # conexos, a nova raíz tenha a cor oposta. Não sei se isso funciona ou se é necessário.
    ultima_cor = 1 # Usado para grafos nao conexos
    while vertices_nao_explorados:
        if len(pilha) == 0:
            menor_label = min([i.label for i in vertices_nao_visitados])
            nova_raiz = grafo.get_vertice(min([i.label for i in vertices_nao_visitados]))
            pilha.append(nova_raiz)
            vertices_nao_visitados.remove(nova_raiz)
            # Colore o grafo com a cor oposta à última cor
            vertices_coloridos[nova_raiz.label] = -ultima_cor
        u = pilha[-1]
        if u not in vizinhos_vertices:
                labels_ordenadas = sorted([i.label for i in grafo.get_adjacentes(u.label)], reverse=True)
                vizinhos_vertices[u] = [grafo.get_vertice(v) for v in labels_ordenadas]
        
        v = vizinhos_vertices[u].pop() if len(vizinhos_vertices[u]) > 0 else None
        if not v:
            pilha.pop()
            vertices_nao_explorados.remove(u) # O vertice eh marcado como explorado
        elif v in vertices_nao_visitados:
            # Se não foi visitado, pinta com a cor oposta ao pai
            vertices_coloridos[v.label] = -vertices_coloridos[u.label]
            vertices_nao_visitados.remove(v)
            pilha.append(v)
            arestas_visitadas.add((u,v))
        else:
            if v in vertices_nao_explorados and pilha[-2] != v: # Checa se é adjaccente na pilha
                if (v,u) not in arestas_visitadas:
                    arestas_visitadas.add((u,v))
                    arestas_de_retorno.add((u,v))
                    # Se ja foi visitado e a sua cor não é compatível com a do vértice que se conecta a ele,
                    #  não é bipartido
                    if vertices_coloridos[u.label] != -vertices_coloridos[v.label] :
                        return False
    return True # Se foi possível fazer toda a coloração, o grafo é bipartido
    

In [459]:
grafo = MatrizAdjacencia("Grafo 1", "./data/GRAFO_BIPARTIDO.txt")
print(f"O grafo eh bipartido: {eh_bipartido(grafo)}")

O grafo eh bipartido: True
